<a href="https://colab.research.google.com/github/LCaravaggio/politext/blob/snowball/Parseo_de_OrdenesdelDia_ListadoCerrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


import json
from google.colab import drive

!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json

drive.mount('/content/drive', force_remount=True)
with open("/content/drive/My Drive/kaggle.json", 'r') as f:
    api_token= json.load(f)

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(api_token, file)

!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "Scraps PDFs.csv"
!kaggle datasets download -d leonardocaravaggio/scraps-pdfs -f "base_formateada_senado_snowball.csv"

Mounted at /content/drive
100% 233M/234M [00:02<00:00, 72.4MB/s]
100% 234M/234M [00:02<00:00, 90.0MB/s]
 92% 132M/143M [00:01<00:00, 98.6MB/s]
100% 143M/143M [00:02<00:00, 69.0MB/s]


In [2]:
import zipfile
import os

os.listdir()

for file in os.listdir():
    if file.endswith('.zip'):
      zip_ref = zipfile.ZipFile(file, 'r')
      zip_ref.extractall()
      zip_ref.close()

In [3]:
# Cargamos la base
base = pd.read_csv('/content/Scraps PDFs.csv')
base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')

<ipython-input-3-e0eeee1141a8>:3: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  base_parseada = pd.read_csv('/content/base_formateada_senado_snowball.csv')


In [4]:
base_parseada.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens'],
      dtype='object')

In [265]:
import re

def generar(l):
  df=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])
  fila=base['link'][base['link'] == l[6]].index[0]
  texto=str(base.loc[fila].text)
  discurso=str(l[3])

  #ordenes = texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')].split('\n')
  #ordenes=['JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTITUCIÓN.','1. ACTAS', '2. REGLAMENTO DEL SENADO', '3. INCOMPATIBILIDADES', '4. COMISIONES MIXTAS', '5. CONVENIOS Y ACUERDOS DE COOPERACIÓN ENTRE COMUNIDADES AUTÓNOMAS']
  patron = r'\d\.\s[A-Z][A-Za-z\s]+'
  ordenes = re.findall(patron, texto[texto.find('ORDEN DEL DÍA'):texto.find('SUMARIO')])
  print(ordenes)


  o=0
  for orden in ordenes:
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(l[2].replace('\n','').replace('\t', '').replace(' ','')+':'+ discurso.replace('\t', '').replace('\n','').replace(' ','')))
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(orden.upper().replace('\n','').replace('\t', '').replace(' ','')))
    #print(texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ',''))
    #print(l[2].replace('\n','').replace('\t', '').replace(' ','')+':'+discurso.replace('\n','').replace('\t', '').replace(' ',''))
    #print('----------')
    ubic_discurso=texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(l[2].replace('\n','').replace('\t', '').replace(' ','')+':'+ discurso.replace('\t', '').replace('\n','').replace(' ',''))
    ubic_titulo=texto[texto.find('SUMARIO'):].replace('\n','').replace('\t', '').replace(' ','').find(orden.replace('\n','').replace('\t', '').replace(' ','').upper())
    if ubic_discurso>ubic_titulo: o+=1

  #print(o-1)
  df['Nombre/Cargo']=[l[2]]
  df['Discurso']=[l[3]]
  df['Cargo']=[l[4]]
  df['Nombre']=[l[5]]
  df['Link']=[l[6]]
  df['Legislatura']=[l[7]]
  df['tipo']=[l[8]]
  df['fecha']=[l[9]]
  df['fecha_formato']=[l[10]]
  df['nombre']=[l[11]]
  df['apellidos']=[l[12]]
  df['procedLiteral']=[l[13]]
  df['procedLugar']=[l[14]]
  df['grupoNombre']=[l[15]]
  df['tokens']=[l[16]]
  try:
    df['orden']=[ordenes[o-1].upper()]
  except:
    df['orden']=[""]


  return df

In [266]:
base2=pd.DataFrame(columns=['Nombre/Cargo', 'Discurso', 'Cargo',
       'Nombre', 'Link', 'Legislatura', 'tipo', 'fecha', 'fecha_formato',
       'nombre', 'apellidos', 'procedLiteral', 'procedLugar', 'grupoNombre',
       'tokens', 'orden'])

In [267]:
for row in base_parseada.values[21:30]:
  base2 = pd.concat([base2, generar(row)], axis=0)

[]
[]
[]
[]
[]
[]
[]
[]
[]


In [256]:
base2

,Nombre/Cargo,Discurso,Cargo,Nombre,Link,Legislatura,tipo,fecha,fecha_formato,nombre,apellidos,procedLiteral,procedLugar,grupoNombre,tokens,orden
0,El señor PARRO DEL PRADO,"Sí, juro.",PARRO DEL PRADO,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,ESTEBAN,PARRO DEL PRADO,Designado,Designado: Asamblea de Madrid,GPP,['jur'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,La señora ORTEGA MORAL,"Sí, juro.",ORTEGA MORAL,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,MARÍA DOLORES,ORTEGA MORAL,Designada,Designada: Parlamento de Andalucía,GPP,['jur'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,La señora POLONIO CONTRERAS,"Sí, prometo.",POLONIO CONTRERAS,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,ROSA LUCÍA,POLONIO CONTRERAS,Designada,Designada: Parlamento de Andalucía,GPS,['promet'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,El señor GARCÍA GARRIDO,"Sí, prometo.",GARCÍA GARRIDO,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,LUIS MANUEL,GARCÍA GARRIDO,Designado,Designado: Parlamento de Andalucía,GPS,['promet'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,La señora DÍAZ PACHECO,"Sí, juro.",DÍAZ PACHECO,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,SUSANA,DÍAZ PACHECO,Designada,Designada: Parlamento de Andalucía,GPS,['jur'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,El señor OSÉS RAMÍREZ,"Sí, juro.",OSÉS RAMÍREZ,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,DANIEL,OSÉS RAMÍREZ,Electo,Electo: La Rioja,GPP,['jur'],JURAMENTO O PROMESA DE ACATAMIENTO A LA CONSTI...
0,El señor QUINTERO CASTAÑEDA,"Muchas gracias, señor presidente. En este inic...",QUINTERO CASTAÑEDA,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,NARVAY,QUINTERO CASTAÑEDA,Electo,Electo: El Hierro,GPMX,"['inici', 'nuev', 'nuev', 'period', 'sesion', ...",2. REGLAMENTO DEL SENADO
0,El señor ANASAGASTI OLABEAGA,"Muchas gracias, señor presidente. Nuestro grup...",ANASAGASTI OLABEAGA,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,IÑAKI MIRENA,ANASAGASTI OLABEAGA,Electo,Electo: Bizkaia,GPV,"['pod', 'men', 'acuerd', 'reform', 'reglament'...",2. REGLAMENTO DEL SENADO
0,El señor CAMACHO SÁNCHEZ,"Muchas gracias, señor presidente. Intervengo e...",CAMACHO SÁNCHEZ,NaN,https://www.senado.es/legis10/publicaciones/pd...,10,PLENOCORTES GENERALES,24 de enero de 2012,2012-01-24,JOSÉ MIGUEL,CAMACHO SÁNCHEZ,Electo,Electo: Toledo,GPS,"['interveng', 'nombr', 'social', 'relacion', '...",2. REGLAMENTO DEL SENADO


In [ ]:
base2=base2.reset_index(drop=True)

In [ ]:
base2['fecha_formato'] = pd.to_datetime(base2['fecha_formato'], errors='coerce')

In [ ]:
from google.colab import files
base2.to_csv('base_formateada_senado_snowball_conorden.csv', escapechar='\\')
files.download('base_formateada_senado_snowball_conorden.csv')